<a href="https://colab.research.google.com/github/avocadopelvis/mental-load/blob/main/mental_load_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
# import libraries
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

In [95]:
# use gpu if available 
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [96]:
# load data
df = pd.read_csv("/content/drive/MyDrive/MENTAL-LOAD-DATASET/final.csv")
# drop first column
df = df.drop(df.columns[0], axis = 1)

# split data into training & testing data
train, test = train_test_split(df, test_size=0.2, random_state = 42)

In [97]:
class SignalDataset(Dataset):
    def __init__(self, df):
      # get x and convert to tensor
      self.x = torch.tensor(df.iloc[:, 0:3840].values)
      # get y and convert to tensor
      self.y = torch.tensor(df["label"].values)
      # get number of samples
      self.n_samples = df.shape[0]

    def __len__(self):
      return self.n_samples

    def __getitem__(self, index):
      return self.x[index], self.y[index]

In [98]:
train_dataset = SignalDataset(train)
test_dataset = SignalDataset(test)

In [99]:
# hyperparameters
input_size = 3840
hidden_size = 2500 
num_classes = 3

epochs = 20
batch_size = 10
learning_rate = 0.001

In [100]:
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True) #,num_workers = 2
test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle=True)

In [101]:
# FULLY CONNECTED NEURAL NETWORK

class NeuralNetwork(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNetwork, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.l2 = nn.Linear(hidden_size, num_classes)
    # self.relu = nn.ReLU()
    # self.l3 = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)
    # out = self.relu(out)
    # out = self.l3(out)
    return out

model = NeuralNetwork(input_size, hidden_size, num_classes)
print(model)

NeuralNetwork(
  (l1): Linear(in_features=3840, out_features=2500, bias=True)
  (relu): ReLU()
  (l2): Linear(in_features=2500, out_features=3, bias=True)
)


In [102]:
# LOSS & OPTIMIZER
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
# optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate, momentum=0.9)

In [103]:
# TRAINING LOOP
n_total_steps = len(train_dataloader)

for epoch in range(epochs):
  for i, data in enumerate(train_dataloader, 0):
    # get the inputs and labels
    inputs, labels = data

    # forward
    outputs = model(inputs.float())
    loss = criterion(outputs, labels)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1)%100 == 0:
      print(f'epoch {epoch+1} / {epochs}, step {i+1} / {n_total_steps}, loss = {loss.item():.4f}')

print("Finished Training")

epoch 1 / 20, step 100 / 168, loss = 1.2215
epoch 2 / 20, step 100 / 168, loss = 1.2909
epoch 3 / 20, step 100 / 168, loss = 0.3311
epoch 4 / 20, step 100 / 168, loss = 0.5267
epoch 5 / 20, step 100 / 168, loss = 0.1789
epoch 6 / 20, step 100 / 168, loss = 0.2306
epoch 7 / 20, step 100 / 168, loss = 0.0394
epoch 8 / 20, step 100 / 168, loss = 0.0108
epoch 9 / 20, step 100 / 168, loss = 0.5236
epoch 10 / 20, step 100 / 168, loss = 0.0051
epoch 11 / 20, step 100 / 168, loss = 0.0006
epoch 12 / 20, step 100 / 168, loss = 0.0153
epoch 13 / 20, step 100 / 168, loss = 1.2819
epoch 14 / 20, step 100 / 168, loss = 0.0117
epoch 15 / 20, step 100 / 168, loss = 1.5076
epoch 16 / 20, step 100 / 168, loss = 0.0481
epoch 17 / 20, step 100 / 168, loss = 0.4875
epoch 18 / 20, step 100 / 168, loss = 0.0000
epoch 19 / 20, step 100 / 168, loss = 0.2261
epoch 20 / 20, step 100 / 168, loss = 0.0090
Finished Training


In [104]:
# TESTING
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  for inputs, labels in test_dataloader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs.float())

    # value, index (We don't need value)
    _, predictions = torch.max(outputs, 1)
    n_samples += labels.shape[0]
    n_correct += (predictions == labels).sum().item()

  acc = 100.0 * n_correct / n_samples
  print(f'accuracy = {acc}')

accuracy = 39.523809523809526


In [105]:
# def train_loop(dataloader, model, loss_fn, optimizer):
#     size = len(dataloader.dataset)
#     for batch, (X, y) in enumerate(dataloader):
#         # Compute prediction and loss
#         pred = model(X.float())
#         loss = loss_fn(pred, y)

#         # Backpropagation
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         if batch % 100 == 0:
#             loss, current = loss.item(), batch * len(X)
#             print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


# def test_loop(dataloader, model, loss_fn):
#     size = len(dataloader.dataset)
#     num_batches = len(dataloader)
#     test_loss, correct = 0, 0

#     with torch.no_grad():
#         for X, y in dataloader:
#             pred = model(X.float())
#             test_loss += loss_fn(pred, y).item()
#             correct += (pred.argmax(1) == y).type(torch.float).sum().item()

#     test_loss /= num_batches
#     correct /= size
#     print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [106]:
# for t in range(num_epochs):
#     print(f"Epoch {t+1}\n-------------------------------")
#     train_loop(train_dataloader, model, criterion, optimizer)
#     test_loop(test_dataloader, model, criterion)
# print("Done!")